In [1]:
from cyecca import lie
import casadi as ca
import math
import scipy.special
from scipy.special import bernoulli

In [2]:
Phi = lie.so3.elem(ca.SX.sym('phi', 3))
phi = ca.norm_2(Phi.param)
scipy.special.bernoulli(0)[-1]

1.0

In [3]:
a = (1/phi)*Phi
a.to_Matrix()

SX(@1=sqrt(((sq(phi_0)+sq(phi_1))+sq(phi_2))), @2=(phi_2/@1), @3=(phi_1/@1), @4=(phi_0/@1), 
[[00, (-@2), @3], 
 [@2, 00, (-@4)], 
 [(-@3), @4, 00]])

In [4]:
def Jr_series(Phi: lie.base.LieAlgebraElement, order: int):
    ad_Phi = Phi.ad()
    S = ca.SX.zeros(3, 3)
    for n in range(order+1):
        S += (-1)**n*ca.mpower(ad_Phi, n)/math.factorial(n+1)
    return S

In [5]:
def Jl_series(Phi: lie.base.LieAlgebraElement, order: int):
    ad_Phi = Phi.ad()
    S = ca.SX.zeros(ad_Phi.shape)
    for n in range(order+1):
        S += ca.mpower(ad_Phi, n)/math.factorial(n+1)
    return S

In [6]:
def Jl_inv_series(Phi: lie.base.LieAlgebraElement, order: int):
    ad_Phi = Phi.ad()
    S= ca.SX.zeros(ad_Phi.shape)
    for n in range(order+1):
        Bn = bernoulli(n)[-1]
        S += Bn*ca.mpower(ad_Phi, n)/math.factorial(n)
    return S

In [7]:
def Jr_inv_series(Phi: lie.base.LieAlgebraElement, order: int):
    ad_Phi = Phi.ad()
    S = ca.SX.zeros(ad_Phi.shape)
    for n in range(order+1):
        Bn = bernoulli(n)[-1]
        S += (-1)**n*Bn*ca.mpower(ad_Phi, n)/math.factorial(n)
    return S

In [8]:
def SO3_Jl_cf(Phi: lie.group_so3.SO3LieAlgebraElement):
    ad_Phi = Phi.ad()
    phi = ca.norm_2(Phi.param)
    cos_phi = ca.cos(phi)
    sin_phi = ca.sin(phi)
    C = ca.if_else(ca.fabs(phi) > 1e-3,
        ca.vertcat(
            (1 - cos_phi)/phi**2,
            (phi - sin_phi)/phi**3),
        ca.vertcat(
            1/2 - phi**2/24 + phi**4/720,
            1/6 - phi**2/120 + phi**4/5040))
    return ca.SX.eye(3) + C[0]*ad_Phi + C[1]*ad_Phi@ad_Phi

In [9]:
def SO3_Jl_inv_cf(Phi: lie.group_so3.SO3LieAlgebraElement):
    ad_Phi = Phi.ad()
    phi = ca.norm_2(Phi.param)
    C = ca.if_else(ca.fabs(phi) > 1e-3,
        (2 - phi/ca.tan(phi/2))/(2*phi**2),
        1/12 + phi**2/720 + phi**4/30240
    )
    return ca.SX.eye(3) - (1/2)*ad_Phi + C*ad_Phi@ad_Phi

This is a singularity in inverse map at 2pi.

In [36]:
Phi = lie.so3.elem(ca.vertcat(2.1*ca.pi, 0, 0))
ca.DM(SO3_Jl_inv_cf(Phi))

DM(
[[1, 00, 00], 
 [00, 20.827, 3.29867], 
 [00, -3.29867, 20.827]])

In [37]:
Phi = lie.so3.elem(ca.vertcat(1.9*ca.pi, 0, 0))
ca.DM(SO3_Jl_inv_cf(Phi))

DM(
[[1, 00, 00], 
 [00, -18.8435, 2.98451], 
 [00, -2.98451, -18.8435]])

In [32]:
def SO3_Jr_cf(Phi: lie.group_so3.SO3LieAlgebraElement):
    # TODO
    pass

In [13]:
def SO3_Jr_inv_cf(Phi: lie.group_so3.SO3LieAlgebraElement):
    # TODO
    pass

In [14]:
ca.DM(Jl_inv_series(lie.so3.elem(ca.vertcat(1, 0, 0)), 10))

DM(
[[1, 0, 0], 
 [0, 0.915244, 0.5], 
 [0, -0.5, 0.915244]])

In [15]:
Phi = lie.so3.elem(ca.vertcat(1*ca.pi, 0, 0))
ca.DM(Jl_series(Phi, 20) @ Jl_inv_series(Phi, 20))

DM(
[[1, 0, 0], 
 [0, 1, -4.04747e-07], 
 [0, 4.04747e-07, 1]])

In [16]:
Phi = lie.so3.elem(ca.vertcat(2*ca.pi, 0, 0))
ca.DM(SO3_Jl_cf(Phi)) #SO3_Jl_inv_cf(Phi))

DM(
[[1, 00, 00], 
 [00, 0, 0], 
 [00, 0, 0]])

In [17]:
ca.DM(Jl_inv_series(lie.so3.elem(ca.vertcat(2*ca.pi, 0, 0)), 10))

DM(
[[1, 0, 0], 
 [0, -10.4993, 3.14159], 
 [0, -3.14159, -10.4993]])

In [18]:
Phi = lie.so3.elem(ca.DM([0.1, 0.2, 0.5]))
Jl_inv_series(Phi, 10) - SO3_Jl_inv_cf(Phi)

SX(@1=-2.58682e-14, @2=-6.47121e-14, @3=-1.29431e-13, 
[[3.75366e-13, @1, @2], 
 [@1, 3.36509e-13, @3], 
 [@2, @3, 6.4726e-14]])

In [19]:
Jl_series(Phi, 10) - SO3_Jl_cf(Phi)

SX(
[[-1.13021e-13, -2.5244e-12, 1.0324e-12], 
 [2.54008e-12, -1.01252e-13, -4.67494e-13], 
 [-9.934e-13, 5.45411e-13, -1.94289e-14]])

In [20]:
Jl_series(Phi, 10) @ Jl_inv_series(Phi, 10)

SX(
[[1, -2.66024e-12, 8.67903e-13], 
 [2.52499e-12, 1, -8.56863e-13], 
 [-1.20619e-12, 1.80189e-13, 1]])

In [21]:
SO3_Jl_cf(Phi) @ SO3_Jl_inv_cf(Phi)

SX(@1=1, 
[[@1, -1.37043e-16, 2.77556e-17], 
 [7.71952e-17, @1, -6.245e-17], 
 [-5.55112e-17, -2.08167e-17, @1]])

In [22]:
Jr_series(Phi, 10) @ Jr_inv_series(Phi, 10)

SX(
[[1, 2.52499e-12, -1.2062e-12], 
 [-2.66024e-12, 1, 1.80203e-13], 
 [8.67903e-13, -8.56863e-13, 1]])

In [23]:
x = ca.SX.sym('x', 3)
x @ x.T

SX(
[[sq(x_0), (x_0*x_1), (x_0*x_2)], 
 [(x_1*x_0), sq(x_1), (x_1*x_2)], 
 [(x_2*x_0), (x_2*x_1), sq(x_2)]])

In [24]:
Jl_inv_series(Phi, 10) - SO3_Jl_inv_cf(Phi)

SX(@1=-2.58682e-14, @2=-6.47121e-14, @3=-1.29431e-13, 
[[3.75366e-13, @1, @2], 
 [@1, 3.36509e-13, @3], 
 [@2, @3, 6.4726e-14]])

In [25]:
import cyecca.symbolic

In [26]:
a = ca.SX.sym('a', 3)
phi = ca.SX.sym('phi')
Phi = lie.so3.elem(phi*a)
Phi

SO3LieAlgebra: SX([(phi*a_0), (phi*a_1), (phi*a_2)])

In [27]:
cyecca.symbolic.casadi_to_sympy(Phi.ad() @ Phi.ad())

Matrix([
[-a_1**2*phi**2 - a_2**2*phi**2,                 a_0*a_1*phi**2,                 a_0*a_2*phi**2],
[                a_0*a_1*phi**2, -a_0**2*phi**2 - a_2**2*phi**2,                 a_1*a_2*phi**2],
[                a_0*a_2*phi**2,                 a_1*a_2*phi**2, -a_0**2*phi**2 - a_1**2*phi**2]])

In [28]:
cyecca.symbolic.casadi_to_sympy(phi**2 * (a @ a.T - a.T @ a * ca.SX.eye(3)))

Matrix([
[phi**2*(-a_1**2 - a_2**2),            a_0*a_1*phi**2,            a_0*a_2*phi**2],
[           a_0*a_1*phi**2, phi**2*(-a_0**2 - a_2**2),            a_1*a_2*phi**2],
[           a_0*a_2*phi**2,            a_1*a_2*phi**2, -phi**2*(a_0**2 + a_1**2)]])

In [29]:
def Jr_inv_series(Phi: lie.base.LieAlgebraElement, order: int):
    ad_Phi = Phi.ad()
    S = ca.SX.zeros(ad_Phi.shape)
    for n in range(order+1):
        Bn = bernoulli(n)[-1]
        S += (-1)**n*Bn*ca.mpower(ad_Phi, n)/math.factorial(n)
    return S